In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, QuantumRegister, ClassicalRegister
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
import qiskit
import copy
import matplotlib.pyplot as plt

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
from sklearn import model_selection, datasets, svm

In [3]:
linnerrud = datasets.load_linnerrud()
X = linnerrud.data[0:100]
Y = linnerrud.target[0:100]
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size = 0.33, random_state = 42)

Traceback (most recent call last):
  Input In [3] in <cell line: 1>
    linnerrud = datasets.load_linnerrud()
AttributeError: module 'sklearn.datasets' has no attribute 'load_linnerrud'

Use %tb to get the full traceback.


In [20]:
print(Y_train)

[0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1
 1 0 1 0 1 1 1 0 1 1 1 1 0 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0 1 1]


In [21]:
print(X_train[0])

[5.2 3.4 1.4 0.2]


In [22]:
N = 4

In [23]:
def feature_map(X):
    q = QuantumRegister(N)
    c = ClassicalRegister(1)
    qc = QuantumCircuit(q,c)

    for i, x in enumerate(X_train[0]):
        qc.rx(x,i)
    
    return qc,c

In [24]:
def variational_circuit(qc, theta):
    for i in range(N-1):
        qc.cnot(i, i+1)
    qc.cnot(N-1, 0)
    for i in range(N):
        qc.ry(theta[i], i)
        
    return qc

In [25]:
def quantum_nn(X, theta, simulator = True):
    qc,c = feature_map(X)

    qc = variational_circuit(qc, theta)

    qc.measure(0,c)

    shots = 1E4
    backend = Aer.get_backend('qasm_simulator')
    
    if simulator == False:
        shots = 5000
        provider = IBMQ.load_account()
        backend = provider.get_backend('ibmq_athens')
        
    job = qiskit.execute(qc, backend, shots = shots)
    result = job.result()
    counts = result.get_counts(qc)
    print(counts)
    
    return(counts['1']/shots)

In [26]:
def loss(prediction, target):
    return (prediction-target)**2

In [27]:
def gradient(X, Y, theta):
    
    delta = 0.01
    grad = []
    for i in range(len(theta)):
        dtheta = copy.copy(theta)
        dtheta += delta
        
        pred_1 = quantum_nn(X, dtheta)
        pred_2 = quantum_nn(X, theta)
        
        grad.append((loss(pred_1, Y) - loss(pred_2, Y))/delta)
        
    return np.array(grad)

In [28]:
def accuracy(X, Y, theta):
    
    counter = 0
    for X_i, Y_i in zip(X, Y):
        prediction = quantum_nn(X_i, theta)
        
        if prediction < 0.5 and Y_i == 0:
            counter += 1
        if prediction >= 0.5 and Y_i == 1:
            counter += 1
            
        return counter/len(Y)

In [ ]:
eta = 0.05

loss_list = []
theta = np.ones(N)

print('Epoch\t Loss\t Training Accuracy')

for i in range(20):
    loss_tmp = []
    
    for X_i, Y_i in zip(X_train, Y_train):
    
        prediction = quantum_nn(X_i, theta)
        loss_tmp.append(loss(prediction, Y_i))
        theta = theta - eta * gradient(X_i, Y_i, theta)
    
    loss_list.append(np.mean(loss_tmp))
    acc = accuracy(X_train, Y_train, theta)
    
    print(f'{i} \t {loss_list[-1]:0.3f} \t {acc:0.3f}')

Epoch	 Loss	 Training Accuracy
{'0': 4608, '1': 5392}
{'0': 4557, '1': 5443}
{'0': 4629, '1': 5371}
{'0': 4578, '1': 5422}
{'0': 4528, '1': 5472}
{'0': 4586, '1': 5414}
{'0': 4519, '1': 5481}
{'1': 5341, '0': 4659}
{'0': 4615, '1': 5385}
{'0': 4534, '1': 5466}
{'1': 5398, '0': 4602}
{'1': 5514, '0': 4486}
{'1': 5476, '0': 4524}
{'0': 4530, '1': 5470}
{'1': 5488, '0': 4512}
{'1': 5423, '0': 4577}
{'1': 5440, '0': 4560}
{'0': 4585, '1': 5415}
{'0': 4470, '1': 5530}
{'1': 5574, '0': 4426}
{'1': 5472, '0': 4528}
{'0': 4354, '1': 5646}
{'1': 5397, '0': 4603}
{'1': 5535, '0': 4465}
{'1': 5462, '0': 4538}
{'1': 5496, '0': 4504}
{'1': 5467, '0': 4533}
{'0': 4487, '1': 5513}
{'0': 4367, '1': 5633}
{'1': 5527, '0': 4473}
{'0': 4457, '1': 5543}
{'0': 4408, '1': 5592}
{'1': 5446, '0': 4554}
{'1': 5573, '0': 4427}
{'0': 4409, '1': 5591}
{'1': 5566, '0': 4434}
{'0': 4514, '1': 5486}
{'1': 5463, '0': 4537}
{'0': 4508, '1': 5492}
{'0': 4536, '1': 5464}
{'0': 4484, '1': 5516}
{'1': 5547, '0': 4453}
{'1

In [ ]:
plt.plot(loss_list)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show

In [ ]:
accuracy(X_test, Y_test, theta)

In [ ]:
clf = svm.SVC()
clf.fit(X_train, Y_train)

In [ ]:
print(clf.predict(X_test))
print(Y_test)

In [ ]:
quantum_nn(X_test[6], theta, simulator = False)

In [ ]:
quantum_nn(X_test[6], theta)

In [ ]:
Y_test[6]